# 14-1 Flask框架

In [1]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

In [1]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello Falsk!"

if __name__ == "__main__":
    app.run()
    print("網站已結束")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jan/2024 14:38:26] "GET / HTTP/1.1" 200 -


網站已結束


In [8]:
from flask import Flask

app = Flask(__name__)
@app.route("/")
def hello():
    return "Hello Flask!"

@app.route("/new", methods = ["GET"])
def name():
    return "Mary"

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Jan/2024 08:05:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 08:05:25] "GET /new HTTP/1.1" 200 -


# 14-2 網頁模板（Template）

In [2]:
from flask import Flask
from flask import render_template

app = Flask(__name__)

@app.route("/")
def index():
    return render_template("index.html")

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Jan/2024 17:28:49] "GET / HTTP/1.1" 200 -


In [1]:
from flask import Flask
from flask import render_template

app = Flask(__name__)

# name = "范振宇"

@app.route("/")
def index():
    return render_template("index2.html", Name = "旗標科技") # 用name也可以

if __name__ == "__main__":
    app.run(debug = False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [3]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route("/")
def index():
    text = [0, 1, 2, 3]
    return render_template("index3.html", text = text)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jan/2024 17:05:40] "GET / HTTP/1.1" 200 -


In [5]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route("/")
def index():
    introduction = ["旗標科技", "台北市中正區 杭州南路一段15-1號19樓", "service@flag.com.tw"]
    return render_template("index4.html", introduction = introduction)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jan/2024 17:09:27] "GET / HTTP/1.1" 200 -


# 14-3 WTForms表單框架

In [1]:
from flask import Flask, render_template, request
from wtforms import Form, StringField, validators

app = Flask(__name__)

class ReviewForm(Form):
    name = StringField("請輸入名字: ", validators = [validators.DataRequired()])

@app.route("/")
def index():
    form = ReviewForm()
    return render_template("index5.html", form = form)

@app.route("/results", methods = ["POST"])
def results():
    form = ReviewForm(request.form)
    if request.method == "POST" and form.validate():
        name = request.form["name"]
        return render_template("results.html", name = name)
    return render_template("index5.html", form = form) # 好像沒影響?

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Jan/2024 23:18:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2024 23:18:42] "GET /favicon.ico HTTP/1.1" 404 -


# 14-4 SQLite資料庫

In [8]:
import sqlite3

connect = sqlite3.connect("database.sqlite")

create_sql = """create table if not exists message_board(
'title' TEXT ,'username' TEXT, "email" TEXT, "message_text" TEXT, "date" TEXT
)"""
connect.execute(create_sql)

In [12]:
import sqlite3

connect = sqlite3.connect("database.sqlite")
connect.execute("""INSERT INTO message_board(
                 title, username, email, message_text, date) VALUES(
                ?, ?, ?, ?, DATETIME('now'))""", 
                ("我", "真", "的", "不知道")) # 這行為要填入的值
connect.commit()
connect.close()

# 14-5 實戰：使用Flask框架建立網頁留言板

In [87]:
from wtforms import Form, StringField, TextAreaField, validators
from wtforms.fields import EmailField
import sqlite3

def sqlite_insert(title, username, email, message_text):
    connect = sqlite3.connect("database.sqlite")
    connect.execute("""INSERT INTO message_board(title, username, 
                    email, message_text, date) VALUES(
                    ?, ?, ?, ?, DATETIME('now'))""", 
                    (title, username, email, message_text))
    connect.commit()
    connect.close()

def sqlite_read():
    conn = sqlite3.connect("database.sqlite")
    create_sql = """create table if not exists message_board(
                    'title' TEXT, 'username' TEXT, 'email' TEXT, 
                    'message_text' TEXT, 'date' TEXT)"""
    conn.execute(create_sql)
    read_sql = "select * from message_board"
    read_data = conn.execute(read_sql)
    dataset = read_data.fetchall()
    conn.close()
    return list(dataset)

class ReviewForm(Form):
    title = StringField("標題:", validators = [validators.DataRequired()])
    username = StringField("作者:", validators = [validators.DataRequired()])
    email = EmailField("email:", validators = [validators.DataRequired()])
    message_text = TextAreaField("留言:", validators = [validators.DataRequired()])
    


In [89]:
from flask import Flask, render_template, request, url_for, redirect
from wtforms import Form, StringField, TextAreaField, validators
from wtforms.fields import EmailField
import sqlite3

def sqlite_insert(title, username, email, message_text):
    connect = sqlite3.connect("database.sqlite")
    connect.execute("INSERT INTO message_board(title, username, email, message_text, date) VALUES(?, ?, ?, ?, DATETIME('now'))", 
                    (title, username, email, message_text))
    connect.commit()
    connect.close()

def sqlite_read():
    conn = sqlite3.connect("database.sqlite")
    create_sql = """create table if not exists message_board(
                    'title' TEXT, 'username' TEXT, 'email' TEXT, 
                    'message_text' TEXT, 'date' TEXT)"""
    conn.execute(create_sql)
    read_sql = "select * from message_board"
    read_data = conn.execute(read_sql)
    dataset = read_data.fetchall()
    conn.close()
    return list(dataset)

class ReviewForm(Form):
    title = StringField("標題:", validators = [validators.DataRequired()])
    username = StringField("作者:", validators = [validators.DataRequired()])
    email = EmailField("email:", validators = [validators.DataRequired()])
    message_text = TextAreaField("留言:", validators = [validators.DataRequired()])

app = Flask(__name__)

@app.route("/", methods = ["GET", "POST"])
def index():
    if request.method == "GET":
        form = ReviewForm()
        says = sqlite_read()
        return render_template("index6.html", says = says, form = form)
    else:
        form = ReviewForm(request.form)
        if request.method == "POST" and form.validate():
            title = request.form["title"]
            username = request.form["username"]
            email = request.form["email"]
            message_text = request.form["message_text"]
            sqlite_insert(title, username, email, message_text)
        return redirect(url_for("index"))

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Feb/2024 20:38:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2024 20:38:32] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [24/Feb/2024 20:38:57] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [24/Feb/2024 20:38:57] "GET / HTTP/1.1" 200 -


# 14-6 實戰：使用Flask框架建立假新聞識別服務

In [1]:
from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators
from sklearn.feature_extraction.text import CountVectorizer
import os
import joblib

cur_dir = os.path.dirname("__file__") # 要""
count_vectorizer = joblib.load(open(os.path.join(cur_dir, "count_vectorizer.pkl"), "rb"))
# count_vectorizer
classifier = joblib.load(open(os.path.join(cur_dir, "classifier.pkl"), "rb"))
# classifier

def classify(document):
    label = {0: "reliable", 1: "unreliable"}
    document_text = count_vectorizer.transform([document])
    y = classifier.predict(document_text)[0]
    # print(classifier.predict_proba(document_text)) # 會出現兩個數值，左邊是reliable，右邊是unreliable
    proba = classifier.predict_proba(document_text).max()
    return label[y], proba

app = Flask(__name__)

class ReviewForm(Form):
    review = TextAreaField("", [validators.DataRequired(), validators.length(min = 15)])

@app.route("/")
def index():
    form = ReviewForm()
    return render_template("reviewform.html", form = form)

@app.route("/results", methods = ["POST"])
def results():
    form = ReviewForm(request.form)
    if request.method == "POST" and form.validate():
        review = request.form["review"]
        label, proba = classify(review)
        return render_template("news_results.html", content = review, 
                               prediction = label, probability = round(proba * 100, 2))
    return render_template("reviewform.html", form = form)

if __name__ == "__main__":
    app.run()

c:\Users\Vick\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Feb/2024 19:57:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2024 19:57:26] "POST /results HTTP/1.1" 200 -


[[0.02962477 0.97037523]]


## 多益題目，先試試看

In [88]:
# from random import randint
from random import sample # sample會回傳list

Questions = [[] for _ in range(7)]
# Questions

for i in range(len(Questions)):
    # print(Questions[i])
    if i == 0:
        Questions[i].append(sample(range(1, 7), 3))
        Questions[i][0].sort()
    if i == 1:
        Questions[i].append(sample(range(7, 32), 5))
        Questions[i][0].sort()
    if i == 2:
        Questions[i].append(sample(range(32, 71, 3), 1))
        Questions[i][0].sort()
    if i == 3:
        Questions[i].append(sample(range(71, 101, 3), 1))
        Questions[i][0].sort()
    if i == 4:
        Questions[i].append(sample(range(101, 131), 5))
        Questions[i][0].sort()
    if i == 5:
        Questions[i].append(sample(range(131, 147, 4), 1))
        Questions[i][0].sort()
    if i == 6:
        Questions[i].append(sample(range(147, 176), 1))
        Questions[i].append(sample(range(176, 186, 5), 1))
        Questions[i].append(sample(range(186, 201, 5), 1))
        Questions[i][0].sort()

for index in range(len(Questions)):
    if index == 6:
        print("Part{}: {}".format(index + 1, Questions[index]))
    else:
        print("Part{}: {}".format(index + 1, Questions[index][0]))

Part1: [2, 5, 6]
Part2: [8, 10, 11, 22, 31]
Part3: [53]
Part4: [77]
Part5: [117, 119, 123, 125, 129]
Part6: [131]
Part7: [[162], [181], [186]]


In [5]:
import requests

response = requests.post("http://127.0.0.1:5000/")
response.text

'<h1>hello world</h1>'

In [7]:
import requests

data = {"name": "vick", "age": 21}
response = requests.post("http://127.0.0.1:5000/", data = data)
print(response.text)

<h1>hello world</h1>
